In [1]:
# import libraries
from pathlib import Path
import string
import pandas as pd
import sys
sys.path.insert(0, '../data_cleaning_and_integration')

from cleaner_lib import remove_puncuations

In [2]:
# set paths
data_p             = Path("../data")
nd_processed_p = data_p / "processed_nextdoor_data"
nd_eth_p = nd_processed_p / "nd_ethnicity_matches.csv"

# out paths for keyword and nd
rt_processed_p = data_p / "processed_reddit_data"
rt_eth_p = rt_processed_p / "reddit_crime_neighborhood_ethnicity.csv"

corpi_p = data_p / 'corpi'
ethnicity_p = corpi_p / 'ethnicity_corpus.csv'

# nodes
node_p = data_p / "neo4j_files/nodes"
relations_p = data_p / "neo4j_files/relationships"
rt_node_p = node_p / "reddit_post_nodes.csv"
nd_node_p = node_p / "nd_post_nodes.csv"

In [3]:
import re
def remove_serialized_list_chars(x):
    return re.sub("\[|\]|'", "", x).split(",")

# Load files

In [4]:
reddit_nodes = pd.read_csv(rt_node_p)
reddit_nodes.head()

,post_id:ID,full_link,title,keywords,LOC,ORG,TIME,:LABEL
0,x4nzh2,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places...,NaN,NaN,san diego,NaN,REDDIT_POST
1,x4ntm7,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost,suddenly appeared; something hard; smoke weed;...,NaN,whaley house; the whaley house,a minute later; late nightearly morning,REDDIT_POST
2,x4n6xv,https://www.reddit.com/r/sandiego/comments/x4n...,language exchange,language exchange; practice spanish; practice;...,NaN,NaN,NaN,REDDIT_POST
3,x4n5aj,https://www.reddit.com/r/SanDiegan/comments/x4...,chula vista police stopping cars going east on...,grand ave; seen; pb; holidays; end; east,NaN,NaN,NaN,REDDIT_POST
4,x4n2rv,https://www.reddit.com/r/SanDiegan/comments/x4...,todd gloria finalizes plan to change park blvd...,zoo uptown; working class; traffic elsewhere; ...,NaN,gtonly,NaN,REDDIT_POST


In [5]:
nd_nodes = pd.read_csv(nd_node_p)
nd_nodes.head()

,post_id:ID,ShortLink,keywords,LOC,ORG,TIME,:LABEL
0,nd1,https://nextdoor.com/p/--3jc5nsXN58?view=detail,willful violation; news trying; means capturin...,NaN,NaN,NaN,NEXTDOOR_POST
1,nd2,https://nextdoor.com/p/--mjpdwdS3yx?view=detail,“ yeah; vehicles chance; shall rebuild; san di...,NaN,like.humidity,only 3 minutes,NEXTDOOR_POST
2,nd3,https://nextdoor.com/p/-3GwdKj4_sMm?view=detail,… enough; water usage; water situation; transi...,NaN,sandags series; angelescarol dellangela,NaN,NEXTDOOR_POST
3,nd4,https://nextdoor.com/p/-4qn3_2yNk_Y?view=detail,public facewithtearsofjoy; faces bios; delete ...,NaN,NaN,NaN,NEXTDOOR_POST
4,nd5,https://nextdoor.com/p/-5-J-BXgJ84y?view=detail,yet nothing; violent felonies; unlawful behavi...,NaN,marston; nextdoor wvideo; dogood; yourself.i; ...,530 pm; morning; night; around midnight last n...,NEXTDOOR_POST


In [6]:
eth_df = pd.read_csv(ethnicity_p)
eth_df["eth_id:ID"] = eth_df.index + 1
eth_df["eth_id:ID"] = "eth" + eth_df["eth_id:ID"].astype("str")
eth_df[":LABEL"] = "ethnicity"
eth_df = eth_df[["eth_id:ID", "ethnicity", ":LABEL"]]
eth_df.drop_duplicates(inplace=True)
eth_df.to_csv(node_p / "ethnicty_nodes.csv", index=False)
eth_df.head()

,eth_id:ID,ethnicity,:LABEL
0,eth1,african american,ethnicity
1,eth2,america,ethnicity
2,eth3,american,ethnicity
3,eth4,arab,ethnicity
4,eth5,asian,ethnicity


In [7]:
nd_df = pd.read_csv(nd_eth_p)
nd_df.head()

,post_id,ShortLink,Author,post_text,post_text_count,Neighborhood,crime,ethnicity
0,nd1,https://nextdoor.com/p/--3jc5nsXN58?view=detail,Hannah Lopez,how late can people be working on construction...,131,Corridor,"['police', 'violation', 'investigative', 'news...",[]
1,nd2,https://nextdoor.com/p/--mjpdwdS3yx?view=detail,Tim Welch,rain has finally arrived in north park but las...,280,Montclair,['money'],[]
2,nd3,https://nextdoor.com/p/-3GwdKj4_sMm?view=detail,News,dont we have a water shortage... jennifer that...,1250,NaN,"['law', 'government', 'illegal', 'homeless', '...",['republican']
3,nd4,https://nextdoor.com/p/-4qn3_2yNk_Y?view=detail,Frank Negrete,guess nd didnt like my question about drinking...,82,Hillcrest Northeast,"['news', 'domestic']",[]
4,nd5,https://nextdoor.com/p/-5-J-BXgJ84y?view=detail,Dawn Burton,day time robbery marston hillsupdate. update u...,1853,Hillcrest Southeast,"['police', 'news', 'laws', 'money', 'law', 'ho...","['black', 'white']"


In [8]:
rt_df = pd.read_csv(rt_eth_p)
# rt_df.drop(columns=["Unnamed: 0", "index"], inplace=True)
rt_df.head()

,subreddit,title,post_id,post_author,post_utc,full_link,post_text,post_text_count,neighborhood,crime,ethnicity
0,sandiego,going to visit san diego next week any places...,x4nzh2,Fearmkultra,2022-09-03 06:57:58+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places ...,12,[],[],[]
1,sandiego,interesting illusion’s,x4ny4c,Break-these-cuffs,2022-09-03 06:55:24+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,interesting illusion’s,2,[],[],[]
2,sandiego,whaley house picture of ghost,x4ntm7,Open_Construction_31,2022-09-03 06:47:09+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost as a kid i saw t...,199,[],['child'],[]
3,sandiego,language exchange,x4n6xv,Poshorock,2022-09-03 06:07:46+00:00,https://www.reddit.com/r/sandiego/comments/x4n...,language exchange is there someone by there wh...,31,[],[],['spanish']
4,SanDiegan,chula vista police stopping cars going east on...,x4n5aj,kaptaincorn,2022-09-03 06:04:54+00:00,https://www.reddit.com/r/SanDiegan/comments/x4...,chula vista police stopping cars going east on...,57,['mission bay'],"['police', 'shooting', 'street', 'investigatio...",[]


# Relationships

In [9]:
eth_id_dict = dict(zip(eth_df.ethnicity, eth_df['eth_id:ID']))
# eth_id_dict

In [10]:
def get_eth_id(eth):
    '''This returns the neighborhood id from neighborhood'''
    try:
        return eth_id_dict[eth.lower()]
    except:
        return 'unknown'

In [11]:
nd_df

,post_id,ShortLink,Author,post_text,post_text_count,Neighborhood,crime,ethnicity
0,nd1,https://nextdoor.com/p/--3jc5nsXN58?view=detail,Hannah Lopez,how late can people be working on construction...,131,Corridor,"['police', 'violation', 'investigative', 'news...",[]
1,nd2,https://nextdoor.com/p/--mjpdwdS3yx?view=detail,Tim Welch,rain has finally arrived in north park but las...,280,Montclair,['money'],[]
2,nd3,https://nextdoor.com/p/-3GwdKj4_sMm?view=detail,News,dont we have a water shortage... jennifer that...,1250,NaN,"['law', 'government', 'illegal', 'homeless', '...",['republican']
3,nd4,https://nextdoor.com/p/-4qn3_2yNk_Y?view=detail,Frank Negrete,guess nd didnt like my question about drinking...,82,Hillcrest Northeast,"['news', 'domestic']",[]
4,nd5,https://nextdoor.com/p/-5-J-BXgJ84y?view=detail,Dawn Burton,day time robbery marston hillsupdate. update u...,1853,Hillcrest Southeast,"['police', 'news', 'laws', 'money', 'law', 'ho...","['black', 'white']"
...,...,...,...,...,...,...,...,...
2806,nd2817,https://nextdoor.com/p/zyBKcPsfG8p4?view=detail,Lisa Busalacchi,got this text today… since i’m expecting some ...,1026,Del Cerro Hearst,"['case', 'security', 'street', 'wrong', 'scam'...",['italy']
2807,nd2818,https://nextdoor.com/p/zzWdg8FDxMw4?view=detail,Eleanor Jacobs,this has to be sketchy scammy don’t click dele...,171,North Park Burlingame/Altadena,"['bad', 'business', 'scam']",[]
2808,nd2819,https://nextdoor.com/p/zzYsgLb5T2sb?view=detail,Rosie Hin,hi everyone. this is my baby 3 ish month old c...,63,University Heights Antique Row N,[],[]
2809,nd2820,https://nextdoor.com/p/zzgTmx49yTM4?view=detail,Grace Joseph,ah yes 1148pm. the perfect time to play the ga...,68,North Park Burlingame/Altadena,"['illegal', 'dangerous']",[]


In [12]:
#nextdoor neighborhood
nd_df = nd_df[nd_df['ethnicity']!='[]']
nd_df['ethnicity'] = nd_df['ethnicity'].apply(lambda x: remove_serialized_list_chars(x))
nd_df = nd_df.explode(column='ethnicity')
# #rename column
nd_df['post_id:START_ID'] = nd_df['post_id']
nd_df[':END_ID'] = nd_df['ethnicity'].apply(lambda x: get_eth_id(x))
nd_df[':TYPE'] = 'ETHNICITY_MENTIONED'
# print(nd_df.head())
#Join to make sure I pick only pos _id from the nodes
nd_df = pd.merge(nd_df, eth_df, left_on=':END_ID', right_on='eth_id:ID')

nd_df = nd_df[['post_id:START_ID',':END_ID',':TYPE']]
nd_df.to_csv(relations_p / 'nextdoor_EM_rels.csv',index=False)
nd_df.head()

/tmp/ipykernel_42049/369752642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nd_df['ethnicity'] = nd_df['ethnicity'].apply(lambda x: remove_serialized_list_chars(x))


,post_id:START_ID,:END_ID,:TYPE
0,nd3,eth45,ETHNICITY_MENTIONED
1,nd55,eth45,ETHNICITY_MENTIONED
2,nd76,eth45,ETHNICITY_MENTIONED
3,nd241,eth45,ETHNICITY_MENTIONED
4,nd766,eth45,ETHNICITY_MENTIONED


In [13]:
#reddit ethnicity
rt_df = rt_df[rt_df['ethnicity']!='[]']
rt_df['ethnicity'] = rt_df['ethnicity'].apply(lambda x: remove_serialized_list_chars(x))
rt_df = rt_df.explode(column='ethnicity')
# #rename column
rt_df['post_id:START_ID'] = rt_df['post_id']
rt_df[':END_ID'] = rt_df['ethnicity'].apply(lambda x: get_eth_id(x))
rt_df[':TYPE'] = 'ETHNICITY_MENTIONED'
# print(nd_df.head())
#Join to make sure I pick only pos _id from the nodes
# nd_crime_neighborhood_df = pd.merge(nd_post_nodes_df, nd_crime_neighborhood_df, left_on='post_id:ID', right_on='post_id:START_ID')

rt_df = pd.merge(rt_df, reddit_nodes, left_on='post_id:START_ID', right_on='post_id:ID')
rt_df = pd.merge(rt_df, eth_df, left_on=':END_ID', right_on='eth_id:ID')

rt_df = rt_df[['post_id:START_ID',':END_ID',':TYPE']]
rt_df.to_csv(relations_p / 'reddit_EM_rels.csv',index=False)
rt_df.head()
# 43273

,post_id:START_ID,:END_ID,:TYPE
0,x4n6xv,eth52,ETHNICITY_MENTIONED
1,x9nh5z,eth52,ETHNICITY_MENTIONED
2,xlefxy,eth52,ETHNICITY_MENTIONED
3,xo8sgr,eth52,ETHNICITY_MENTIONED
4,xirlqt,eth52,ETHNICITY_MENTIONED


In [14]:
reddit_nodes

,post_id:ID,full_link,title,keywords,LOC,ORG,TIME,:LABEL
0,x4nzh2,https://www.reddit.com/r/sandiego/comments/x4n...,going to visit san diego next week any places...,NaN,NaN,san diego,NaN,REDDIT_POST
1,x4ntm7,https://www.reddit.com/r/sandiego/comments/x4n...,whaley house picture of ghost,suddenly appeared; something hard; smoke weed;...,NaN,whaley house; the whaley house,a minute later; late nightearly morning,REDDIT_POST
2,x4n6xv,https://www.reddit.com/r/sandiego/comments/x4n...,language exchange,language exchange; practice spanish; practice;...,NaN,NaN,NaN,REDDIT_POST
3,x4n5aj,https://www.reddit.com/r/SanDiegan/comments/x4...,chula vista police stopping cars going east on...,grand ave; seen; pb; holidays; end; east,NaN,NaN,NaN,REDDIT_POST
4,x4n2rv,https://www.reddit.com/r/SanDiegan/comments/x4...,todd gloria finalizes plan to change park blvd...,zoo uptown; working class; traffic elsewhere; ...,NaN,gtonly,NaN,REDDIT_POST
...,...,...,...,...,...,...,...,...
43268,scdqum,https://www.reddit.com/r/UCSD/comments/scdqum/...,la jolla donor makes 50m research t that could...,wasnt aware; san diego; never wondered; good s...,NaN,usc the san diego uniontribune i; usc,NaN,REDDIT_POST
43269,sca7fv,https://www.reddit.com/r/UCSD/comments/sca7fv/...,new covid variant detected in at least 40 diff...,sigma variant; new shot; like omicron; kill li...,NaN,NaN,NaN,REDDIT_POST
43270,sc9b5t,https://www.reddit.com/r/sandiego/comments/sc9...,tmz baltimore maggots leaked video twitter sca...,NaN,NaN,tmz baltimore,NaN,REDDIT_POST
43271,sc90i4,https://www.reddit.com/r/UCSD/comments/sc90i4/...,mailing services while school’s online,thing thankfully; theyre forwarding; pretty im...,NaN,clownface,NaN,REDDIT_POST


# Command to load data
* Assumes node dir and rels dir in import/
```bash
bin/neo4j-admin import --force --multiline-fields=true --nodes=import/nodes/crime_nodes.csv import/nodes/ethnicty_nodes.csv import/nodes/nd_post_nodes.csv import/nodes/neighborhood_nodes.csv import/nodes/pd_crime_nodes.csv import/nodes/reddit_post_nodes.csv --relationships=import/rels/nextdoor_EM_rels.csv import/rels/nextdoor_HI_rels.csv import/rels/reddit_HI_rels.csv import/rels/reddit_EM_rels.csv
```